### KAYAK

In [2]:
import requests
import pandas as pd
import json
import os
from collections import Counter
import numpy as np

In [3]:
cities = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg",
          "Chateau du Haut Koenigsbourg","Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon",
          "Gorges du Verdon","Bormes les Mimosas","Cassis","Marseille","Aix en Provence","Avignon","Uzes",
          "Nimes","Aigues Mortes","Saintes Maries de la mer","Collioure","Carcassonne","Ariege","Toulouse",
          "Montauban","Biarritz","Bayonne","La Rochelle"]

In [54]:
# Test 

city_name = 'Bayeux,FR'
API_key= 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

def get_weather(API_key, city, return_=False):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_key}"
    r = requests.get(url).json()
    print(f"Ville :\n {r['name']}\n")
    print(f"Coordonnées GPS :\n Longitude : {r['coord']['lon']}\n Latitude : {r['coord']['lat']}\n")
    print(f"Conditions Météorologiques :\n Temperature(C°) : {int(r['main']['temp'] - 273.15)} \n Ressenti(C°) : {int(r['main']['feels_like'] - 273.15)} \n Humidité : {r['main']['humidity']}")
    print(f" Météo : {r['weather'][0]['main']}")
    if return_:
        return r

In [55]:
get_weather(API_key, city_name)

Ville :
 Arrondissement de Bayeux

Coordonnées GPS :
 Longitude : -0.8333
 Latitude : 49.25

Conditions Météorologiques :
 Temperature(C°) : 15 
 Ressenti(C°) : 14 
 Humidité : 74
 Météo : Clouds


In [56]:
cols = ['city_id' , 'name', 'latitude', 'longitude']
url = 'https://nominatim.openstreetmap.org/search'
n = len(cities)

In [57]:
s = []
for i in range(n):
    params = {"city": cities[i], "country": "France", "format": "json"}
    r = requests.get(url, params=params).json()
    s.append(r)

In [58]:
for i in range(len(s)):
    if s[i] == []:
        print(i, cities[i])

17 Gorges du Verdon
29 Ariege


Ces 2 villes n'ont pas d'informations

In [59]:
cities_id = [i for i in range(n) if s[i] != []]
cities_name = [city for i,city in enumerate(cities) if s[i] != []]
cities_lat = [s[i][0]['lat'] for i in range(n) if s[i] != []]
cities_lon = [s[i][0]['lon'] for i in range(n) if s[i] != []]

rows = [cities_id, cities_name, cities_lat, cities_lon]

In [60]:
df = pd.DataFrame({cols[0] : cities_id, cols[1] : cities_name, cols[2] : cities_lat, cols[3] : cities_lon})
df.head()

,city_id,name,latitude,longitude
0,0,Mont Saint Michel,48.6359541,-1.511459954959514
1,1,St Malo,48.649518,-2.0260409
2,2,Bayeux,49.2764624,-0.7024738
3,3,Le Havre,49.4938975,0.1079732
4,4,Rouen,49.4404591,1.0939658


In [61]:
df.to_csv('kayak_df.csv', index=None)

In [62]:
df = pd.read_csv('kayak_df.csv')
df.head()

,city_id,name,latitude,longitude
0,0,Mont Saint Michel,48.635954,-1.511460
1,1,St Malo,48.649518,-2.026041
2,2,Bayeux,49.276462,-0.702474
3,3,Le Havre,49.493898,0.107973
4,4,Rouen,49.440459,1.093966


In [63]:
cols = ['city_id', 'weather', 'rain_probability', 'temperature_day', 'humidity', 'pressure']

In [64]:
df2 = pd.DataFrame(columns=cols)

url = "https://api.openweathermap.org/data/2.5/onecall"

s= {}

for i in range(len(cities_id)):

    params = {
            "lat": df.loc[i,'latitude'],
            "lon": df.loc[i,'longitude'],
            "exclude": "current, minutely, hourly", # Exclusion de certains parametres de  l'API: current, minutely and hourly
            "units": "metric", 
            "appid": "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX", # API key
        }

    r2 = requests.get(url,params).json()
    n = len(r2['daily'])
    count,mean,mode,pressure,humidity = 0,0,[],0,0
    
    for el in r2['daily']:
        pressure += el['pressure']
        humidity += el['humidity']
        
        if el['weather'][0]['main'] == 'Rain':   # 'pop'  : probabilité de pluie
            count += el['pop']*el['rain']       
            
        mean += el['temp']['day']
        mode.append(el['weather'][0]['main'])
   
    pressure /= n
    humidity /= n
    mean /= n
    s[df.loc[i,'name']] = Counter(mode)
    mode = Counter(mode).most_common(1)[0][0]
    
    df2.loc[i, 'city_id'] = i
    df2.loc[i,'weather']= mode
    df2.loc[i,'rain_probability']= round(count,2)
    df2.loc[i,'temperature_day']= int(mean)
    df2.loc[i, 'pressure'] = round(pressure,2)
    df2.loc[i, 'humidity'] = round(humidity,2)
 

In [65]:
rain, clouds, clear = [], [], []
for k,v in s.items():
    rain_count, clouds_count, clear_count = 0,0,0
    a = list(v.elements())
    for el in a:
        if el == 'Rain':
            rain_count += 1
        if el == 'Clouds':
            clouds_count += 1
        if el == 'Clear':
            clear_count += 1
    rain.append(rain_count)
    clouds.append(clouds_count)
    clear.append(clear_count)

In [96]:
new_df = pd.DataFrame({'Rain' : rain, 'Clouds' : clouds, 'Clear' : clear})
new_df.head()

,Rain,Clouds,Clear
0,4,4,0
1,5,3,0
2,5,3,0
3,6,2,0
4,5,3,0


In [97]:
df_final = pd.concat([df,df2,new_df], axis=1).drop(['city_id'],axis=1)

# Calcul du rang Meteo : 0.2 * Pluvieux + 0.5 * Nuageux + 1 * Ciel_dégagé
df_final['rank_weather'] = df_final['Rain']*0.2 + df_final['Clouds']*0.5 + df_final['Clear']*1

df_final['rank_weather'] = df_final['rank_weather'].rank(method='max').astype(int)
df_final['rank_rain'] = df_final['rain_probability'].rank(method='min').astype(int)
df_final['rank_temp'] = df_final['temperature_day'].rank(method='max').astype(int)
df_final['rank_humi'] = df_final['humidity'].rank(method='min').astype(int)
df_final['rank_pres'] = df_final['pressure'].rank(method='max').astype(int)


# Calcul final du rang : 0.5 * rang_pluie + 0.35 * rang_meteo + 0.3 * rang_temperature + 0.2 * rang_humidité + 0.2 * rang_pression_atmospherique / nombre de rangs
df_final['rank_'] = (df_final['rank_rain']*0.5 + df_final['rank_weather']*0.35 + df_final['rank_temp']*0.3 + df_final['rank_humi']*0.2 + df_final['rank_pres']*0.2)/5
df_final = df_final.sort_values(by='rank_').drop(df_final.columns[-6:],axis=1)
df_final['rank'] = [i+1 for i in range(len(df_final))]
df_final = df_final.reset_index().drop(['index'],axis=1)
df_final.head(5)

,name,latitude,longitude,weather,rain_probability,temperature_day,humidity,pressure,Rain,Clouds,Clear,rank
0,Rouen,49.440459,1.093966,Rain,9.11,18,62.0,1015.75,5,3,0,1
1,St Malo,48.649518,-2.026041,Rain,8.55,17,67.62,1016.38,5,3,0,2
2,Bayeux,49.276462,-0.702474,Rain,11.88,17,66.5,1015.88,5,3,0,3
3,Chateau du Haut Koenigsbourg,48.249523,7.345492,Rain,9.88,14,64.25,1016.62,4,3,1,4
4,Strasbourg,48.584614,7.750713,Rain,8.09,16,66.38,1016.88,4,3,1,5


Les 5 meilleures villes sont : 

In [98]:
df_final['name'][:5]

0                           Rouen
1                         St Malo
2                          Bayeux
3    Chateau du Haut Koenigsbourg
4                      Strasbourg
Name: name, dtype: object

In [99]:
df_final.to_csv('kayak_final.csv', index=False)